# Tests for `newrelic_sb_sdk.utils.response`

## Imports

In [ ]:
import json
from typing import Any, Dict, Union

In [ ]:
from deepdiff import DeepDiff

In [ ]:
from newrelic_sb_sdk.utils.response import get_response_data, print_response

## Tests

In [ ]:
class MockedResponse:
    def __init__(
        self, *, json_data: Union[Dict[str, Any], None] = None, status_code: int = 200
    ):
        self.json_data = json_data
        self.status_code = status_code

    def json(self) -> Union[Dict[str, Any], None]:
        return self.json_data

In [ ]:
class TestUtilsResponse:
    json_data = {
        "data": {
            "actor": {
                "user": {
                    "email": "admin@example.com",
                },
                "accounts": [
                    {
                        "id": 1234567,
                        "name": "Account 1234567",
                    },
                ],
            },
            "custom_action": {"results": ["item_0", "item_1"]},
        },
    }

    def test_print_response(self, capfd):
        response = MockedResponse(json_data=self.json_data)
        print_response(response)
        out, _ = capfd.readouterr()

        assert out.strip() == json.dumps(self.json_data, indent=4).strip()  # nosec

    def test_print_response_compact(self, capfd):
        response = MockedResponse(json_data=self.json_data)
        print_response(response, compact=True)
        out, _ = capfd.readouterr()

        assert out.strip() == json.dumps(self.json_data, indent=None).strip()  # nosec

    def test_get_response_data_without_key_path(self):
        response = MockedResponse(json_data=self.json_data)
        response_data = get_response_data(response)

        assert not DeepDiff(self.json_data["data"]["actor"], response_data)  # nosec

    def test_get_response_data_with_key_path(self):
        response = MockedResponse(json_data=self.json_data)
        response_data = get_response_data(response, key_path="accounts:0:name")

        assert not (  # nosec
            DeepDiff(
                self.json_data["data"]["actor"]["accounts"][0]["name"], response_data
            )
        )

    def test_get_response_data_with_custom_action(self):
        response = MockedResponse(json_data=self.json_data)
        response_data = get_response_data(
            response, key_path="results", action="custom_action"
        )

        assert not (  # nosec
            DeepDiff(self.json_data["data"]["custom_action"]["results"], response_data)
        )